In [1]:
!pip install pandas nltk gensim scikit-learn tensorflow==2.11.0


ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: none)
ERROR: No matching distribution found for tensorflow==2.11.0



  Using cached gensim-4.3.3.tar.gz (23.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import numpy as np

# Memuat dataset
file_path = '/mnt/data/comments_data.csv'
data = pd.read_csv(file_path)

# Mengambil kolom 'Comment' dan menampilkan 5 baris pertama
comments = data['Comment']
comments.head()
